In [30]:
import json
import base64
import random
import requests
import numpy as np
from PIL import Image
from datetime import datetime
from cryptography.fernet import Fernet
from math import radians, cos, sin, asin, sqrt

def generate_key():
    return Fernet.generate_key()

# Encrypt a message using symmetric encryption
def encrypt_message(message, key):
    cipher = Fernet(key)
    return cipher.encrypt(message.encode())

def decrypt_message(ciphertext, key):
    cipher = Fernet(key)
    return cipher.decrypt(ciphertext).decode()

    # Calculate the great circle distance between two points on the earth (km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

def ip_api():
    url = 'http://ip-api.com/json/'
    try:
        data = requests.get(url).json()
        lat, lon = float(data['lat']), float(data['lon'])
        return {'service': 'ip-api.com', 'lat': lat, 'lon': lon}
    except Exception as e:
        print(f"ip-api.com error: {e}")
        return None

def ipgeolocation_io():
    API_KEY = 'd3e18956ee4f4d9fb188ac5395be075e'  # <-- Insert your ipgeolocation.io API key here
    url = f'https://api.ipgeolocation.io/ipgeo?apiKey={API_KEY}'
    try:
        data = requests.get(url).json()
        lat, lon = float(data['latitude']), float(data['longitude'])
        return {'service': 'ipgeolocation.io', 'lat': lat, 'lon': lon}
    except Exception as e:
        print(f"ipgeolocation.io error: {e}")
        return None

def ipstack_location():
    API_KEY = 'd9deefe9b26eb930cac19065ac2ac08f'  # <-- Insert your ipstack.com API key here
    url = f'http://api.ipstack.com/check?access_key={API_KEY}'
    try:
        data = requests.get(url).json()
        lat, lon = float(data['latitude']), float(data['longitude'])
        return {'service': 'ipstack.com', 'lat': lat, 'lon': lon}
    except Exception as e:
        print(f"ipstack.com error: {e}")
        return None
    
def get_coordinates_data():
    services = [
        ('ip-api.com', ip_api),
        ('ipgeolocation.io', ipgeolocation_io),
        ('ipstack.com', ipstack_location)
    ]
    results = {}
    for name, func in services:
        try:
            loc = func()
            if loc:
                results[name] = {
                    'lat': loc['lat'],
                    'lon': loc['lon']
                }
            else:
                results[name] = {'error': 'No data'}
        except Exception as e:
            results[name] = {'error': str(e)}
    return results

def get_location_data():
    url = 'http://ip-api.com/json/'
    try:
        response = requests.get(url)
        data = response.json()
        return {
            'city': data.get('city', ''),
            'region': data.get('regionName', ''),
            'country': data.get('country', ''),
            'coordinates': f"{data.get('lat', '')},{data.get('lon', '')}",
            'more_coordinates': get_coordinates_data(),
            'extra': {
                'timezone': data.get('timezone', ''),
                'isp': data.get('isp', ''),
                'org': data.get('org', ''),
                'as': data.get('as', '')
            }
        }
    except Exception as e:
        print(f"Location error: {e}")
        return None

## LSB Encoder

In [31]:
def lsb_encode(img_path: str, secret_message: str, key: bytes, output_path: str) -> None:
    try:
        enc_bytes = encrypt_message(secret_message, key)
        enc_b64 = base64.urlsafe_b64encode(enc_bytes).decode()
        enc_b64 += chr(0)
        img = Image.open(img_path)
        encoded = img.copy()
        width, height = img.size
        data = ''.join(f"{ord(c):08b}" for c in enc_b64)
        idx = 0
        for y in range(height):
            for x in range(width):
                pixel = list(img.getpixel((x, y)))
                for n in range(3):
                    if idx < len(data):
                        pixel[n] = pixel[n] & ~1 | int(data[idx])
                        idx += 1
                encoded.putpixel((x, y), tuple(pixel))
                if idx >= len(data):
                    encoded.save(output_path)
                    return
        raise ValueError("Message too large to encode in image.")
    except Exception as e:
        print(f"Error during encoding: {e}")

## LSB Decoder

In [32]:
def lsb_decode(img_path: str, key: bytes) -> str:
    try:
        img = Image.open(img_path)
        width, height = img.size
        bits = []
        for y in range(height):
            for x in range(width):
                pixel = img.getpixel((x, y))
                for n in range(3):
                    bits.append(str(pixel[n] & 1))
        chars = [chr(int(''.join(bits[i:i+8]), 2)) for i in range(0, len(bits), 8)]
        message = ''.join(chars)
        enc_b64 = message.split(chr(0))[0]
        enc_bytes = base64.urlsafe_b64decode(enc_b64)
        return decrypt_message(enc_bytes, key)
    except Exception as e:
        print(f"Error during decoding: {e}")
        return None

## Usage example

In [33]:
inputPath = "assets/images/raw.png"
outputPath = "assets/images/lsb-encoded.png"

secret_data = {
    "message": "Very Secret Message",
    "date": datetime.now().isoformat(),
    "location": get_location_data()
}

secret_message = json.dumps(secret_data, indent=4)

key = generate_key()
print("Generated encryption key (store this securely):")
print(key.decode())

# To encode:
lsb_encode(inputPath, secret_message, key, outputPath)

# To decode:
message = lsb_decode(outputPath, key)
print("\nDecoded message:")
print(message)

Generated encryption key (store this securely):
x6O0PdIoFZiwNQbu-MLhIhyArd2u2sItuI0pnPm5wYg=

Decoded message:
{
    "message": "Very Secret Message",
    "date": "2025-06-15T16:59:36.181512",
    "location": {
        "city": "Laurierville",
        "region": "Quebec",
        "country": "Canada",
        "coordinates": "46.3116,-71.6839",
        "more_coordinates": {
            "ip-api.com": {
                "lat": 46.3116,
                "lon": -71.6839
            },
            "ipgeolocation.io": {
                "lat": 46.20712,
                "lon": -71.61426
            },
            "ipstack.com": {
                "lat": 46.37099838256836,
                "lon": -71.61199951171875
            }
        },
        "extra": {
            "timezone": "America/Toronto",
            "isp": "Sogetel Inc",
            "org": "Sogetel Inc",
            "as": "AS4540 SOGETEL INC"
        }
    }
}
